# 词元嵌入

In [2]:
import torch
from torch import nn

In [3]:
input_ids = torch.tensor([2,3,5,1])

In [4]:
vocab_size = 6
output_dim = 3

In [5]:
torch.manual_seed(123)
embedding_layer = nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [6]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


# 位置编码

In [7]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = nn.Embedding(vocab_size, output_dim)


In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
from importlib.metadata import version
import tiktoken

class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.inputs = []
        self.targets = []
        enc_ids = tokenizer.encode(text)
        for i in range(0, len(enc_ids) - max_length, stride):
            input_ids = enc_ids[i : i + max_length]
            target_ids = enc_ids[i + 1 : i + max_length + 1]
            self.inputs.append(torch.tensor(input_ids))
            self.targets.append(torch.tensor(target_ids))
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]


In [13]:
def create_dataloader_v1(text, batch_size, max_length = 256, stride = 128, shuffle = True, drop_last = True, num_workers = 0):
    tokenizer = tiktoken.get_encoding("gpt2");
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = shuffle, drop_last = drop_last, num_workers = num_workers)
    return dataloader

In [14]:
with open("./../the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length = max_length, stride = max_length, shuffle = False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInput shape:", inputs.shape)


Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Input shape: torch.Size([8, 4])


In [15]:
print(targets.shape)

torch.Size([8, 4])


In [16]:
token_embeddings = token_embedding_layer(inputs)
print("\nToken Embeddings shape:", token_embeddings.shape)


Token Embeddings shape: torch.Size([8, 4, 256])


In [17]:
print(token_embeddings[0])

tensor([[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
        [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
        [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
        [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],
       grad_fn=<SelectBackward0>)


In [ ]:
print(token_embeddings[0].shape)

torch.Size([4, 256])


In [20]:
context_length = max_length
pos_embedding_layer = nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [21]:
input_embeddings = token_embeddings + pos_embeddings
print("\nInput Embeddings shape:", input_embeddings.shape)


Input Embeddings shape: torch.Size([8, 4, 256])


In [22]:
torch.arange(context_length)

tensor([0, 1, 2, 3])